In [679]:

import pandas as pd 
import re
import emoji
import warnings
warnings.filterwarnings('ignore')


In [680]:
df = pd.read_excel("train_nor_811 (1).xlsx")
df.drop("Unnamed: 0",axis =1, inplace=True)
df

,Emotion,Sentence
0,Other,1423k cho mình xin 😃bài nhạc tên là gì với ạ
1,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2,Disgust,lo học đi . yêu đương lol gì hay lại thích học...
3,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này :))
4,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...
...,...,...
12340,Surprise,Tôi không bao giờ từng nghĩ rằng bạn sẽ đạt đư...
12341,Surprise,Tôi cảm thấy thực sự vui mừng với sự thành côn...
12342,Surprise,Tôi không thể nghĩ ra một cách nào để mô tả cả...
12343,Surprise,Tôi thật sự rất ngạc nhiên với điều đó.


In [681]:
df['Emotion'].value_counts()

Surprise     1839
Other        1804
Anger        1772
Sadness      1763
Enjoyment    1751
Disgust      1727
Fear         1689
Name: Emotion, dtype: int64

In [682]:
# df['Emotion'] = df['Emotion'].replace('Disgust', 'Anger')

In [683]:

# Remove duplicates from the "Sentence" column
df = df.drop_duplicates(subset=["Sentence"])

print(df)

         Emotion                                           Sentence
0          Other       1423k cho mình xin 😃bài nhạc tên là gì với ạ
1        Disgust  cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2        Disgust  lo học đi . yêu đương lol gì hay lại thích học...
3      Enjoyment    uớc gì sau này về già vẫn có thể như cụ này :))
4      Enjoyment  mỗi lần có video của con là cứ coi đi coi lại ...
...          ...                                                ...
12340   Surprise  Tôi không bao giờ từng nghĩ rằng bạn sẽ đạt đư...
12341   Surprise  Tôi cảm thấy thực sự vui mừng với sự thành côn...
12342   Surprise  Tôi không thể nghĩ ra một cách nào để mô tả cả...
12343   Surprise            Tôi thật sự rất ngạc nhiên với điều đó.
12344   Surprise  Tôi cảm thấy hạnh phúc vì bạn đã làm được điều...

[12339 rows x 2 columns]


In [684]:
nan_count = df['Sentence'].isna().sum()

print("Number of NaN values in 'emotion' column:", nan_count)

Number of NaN values in 'emotion' column: 1


In [685]:
# Count the number of NaN values in the 'text' column
df = df.dropna(subset=['Sentence'])



In [686]:
# df= df[df['Emotion'] != 'Other']

# df.tail()

In [687]:
df['Emotion'].value_counts()

Surprise     1837
Other        1803
Anger        1772
Sadness      1763
Enjoyment    1751
Disgust      1725
Fear         1687
Name: Emotion, dtype: int64

# Transform emoticon to text ####

In [2]:
emoticon_to_word = {
    "@@": "ngạc nhiên ",
    "^^": "happy ",
    '(°°)': "bối rồi",

}

In [3]:
def transform_emoticons(text):
    if not isinstance(text, str):
        # Handle the case where input is not a string (e.g., float)
        return str(text)  # Convert the input to a string
    
    for emoticon, word in emoticon_to_word.items():
        text = text.replace(emoticon, word)
    return text

# Example usage
input_text = "I'm feeling so happy! 😃 :DDD :] ;) @@"
output_text = transform_emoticons(input_text)
print(output_text)



I'm feeling so happy! 😃 :DDD :] ;) ngạc nhiên 


In [690]:
df['Sentence']= df['Sentence'].apply(transform_emoticons)

df.head()

,Emotion,Sentence
0,Other,1423k cho mình xin 😃bài nhạc tên là gì với ạ
1,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2,Disgust,lo học đi . yêu đương lol gì hay lại thích học...
3,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này :))
4,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...


# 1. Standardizing words.


In [691]:

def remove_repeated_sequences(text):
    # Define a regex pattern to find sequences of repeated characters (including punctuation)
    pattern = r'(\S)\1+'
    
    # Replace the repeated sequences with a single instance of the character
    modified_text = re.sub(pattern, r'\1', text, flags=re.IGNORECASE)
    
    return modified_text

# Example
text1 = "hahaaaa"
result1 = remove_repeated_sequences(text1)
print(result1)  

text2 = ":]]]]"
result2 = remove_repeated_sequences(text2)
print(result2) 

text3 = "WooOOOOW!!!"
result3 = remove_repeated_sequences(text3)
df['Sentence']= df['Sentence'].apply(remove_repeated_sequences)

print(result3)


haha
:]
WoW!


## Extract all emoji have in data


In [692]:


def extract_unique_emojis_from_column(sentences):
    emoji_pattern = re.compile("[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002600-\U000027BF]+")
    
    all_emojis = []
    for sentence in sentences:
        emojis = re.findall(emoji_pattern, sentence)
        all_emojis.extend(emojis)
    
    unique_emojis = list(set(all_emojis))
    return unique_emojis

# Example list of sentences in a column
sentences_column = [
    "Hello! 😄🌍🌟 Another day, another emoji! 😄🌍",
    "Emojis make communication fun! 🎉😊",
    "I love coding with emojis! 👩‍💻💡"
]

emojis_list = extract_unique_emojis_from_column(df["Sentence"])
print(emojis_list)

['😤', '😡🤬', '😗😚', '🌊', '😞🙁', '😨😮', '😮🤩', '✈', '😁😍💥', '😁😱', '😲😍💕', '😍', '🤯', '🦱', '😱🤯', '😱😂', '🍚😫', '🤨🤫', '😮😊', '☹', '😍🤩', '😲🤩🥰', '😲🥳', '😰💔', '😱😳🤔', '😁😂🤣', '🐕', '🚮😬', '😍😲🤯', '😖', '🌪', '😍😲🤪', '😮😶😑', '🙈😫', '😛', '🏚', '😢😲', '😲😍🤗', '😱😢', '😚😘', '😄😮', '😩🙏🤕', '🙂', '😭🤢', '🤑', '🌄', '😍😢😲', '😲😳🤩', '😯😃', '😄', '🌸', '🤣👍', '🌫', '🐶😂', '🏠🤮', '🤒', '😆😁', '😧', '👏🏻👏🏻👏🏻', '😲😍👏', '🥴🤢', '😮🌟', '😱😳😍', '😲😁🎉', '🕸😂', '😮🤯😱', '😱😅', '💩', '😑😆❤', '🙈🤢', '🌑', '😍😁🥳', '😠😡', '☠', '💥', '🌕', '🙃😨', '😰😓', '🤮💀', '😍😲🥳', '😠😤', '😂', '💃', '🤧', '👶💔', '🤦🏻', '😵😲', '😮😍', '🤐🤢', '😡😤', '😳😮', '😂😳🙀', '😭😮', '💥😠', '😁😂', '😳😍😲', '😲🤩', '😁👍😲', '😊', '😒', '😂😁', '😉', '😱🤢', '💣😨', '💪', '😲😱🤔', '😞🤣', '♂', '🎨', '😃😮', '🌍', '☻', '😂💔', '😮⚡', '😞😄', '😌', '😳😆', '😰💪', '😁🤩', '😳😵🌪', '📱😫', '😱😔😩', '✌', '😢😞', '🤧😣', '🚪😨', '😊😮', '🙁', '🍕', '😠😣', '😰😞', '😱😂🤪', '😮👦👧', '😂😍', '😲😍🤩', '😧😃', '⚡', '😈', '😮👀🤷', '😃😲', '😥', '😚', '🌙', '😮😲', '😟😰', '😵💦', '😨😔', '😨🙅', '🦁', '😲😮', '😶😲', '😁😲', '😡', '🙀', '🤣🤭', '🤤', '😠🤬', '😔😞', '🤔😲', '💕', '🍔', '😆😮', '😳😲🤔', '😮🤔😯', '😵😮', '😨🤔', '🥳', '😲😍👍', '😴😲', '

In [693]:

individual_emojis = [emoji for sublist in emojis_list for emoji in sublist]

print(individual_emojis)

['😤', '😡', '🤬', '😗', '😚', '🌊', '😞', '🙁', '😨', '😮', '😮', '🤩', '✈', '😁', '😍', '💥', '😁', '😱', '😲', '😍', '💕', '😍', '🤯', '🦱', '😱', '🤯', '😱', '😂', '🍚', '😫', '🤨', '🤫', '😮', '😊', '☹', '😍', '🤩', '😲', '🤩', '🥰', '😲', '🥳', '😰', '💔', '😱', '😳', '🤔', '😁', '😂', '🤣', '🐕', '🚮', '😬', '😍', '😲', '🤯', '😖', '🌪', '😍', '😲', '🤪', '😮', '😶', '😑', '🙈', '😫', '😛', '🏚', '😢', '😲', '😲', '😍', '🤗', '😱', '😢', '😚', '😘', '😄', '😮', '😩', '🙏', '🤕', '🙂', '😭', '🤢', '🤑', '🌄', '😍', '😢', '😲', '😲', '😳', '🤩', '😯', '😃', '😄', '🌸', '🤣', '👍', '🌫', '🐶', '😂', '🏠', '🤮', '🤒', '😆', '😁', '😧', '👏', '🏻', '👏', '🏻', '👏', '🏻', '😲', '😍', '👏', '🥴', '🤢', '😮', '🌟', '😱', '😳', '😍', '😲', '😁', '🎉', '🕸', '😂', '😮', '🤯', '😱', '😱', '😅', '💩', '😑', '😆', '❤', '🙈', '🤢', '🌑', '😍', '😁', '🥳', '😠', '😡', '☠', '💥', '🌕', '🙃', '😨', '😰', '😓', '🤮', '💀', '😍', '😲', '🥳', '😠', '😤', '😂', '💃', '🤧', '👶', '💔', '🤦', '🏻', '😵', '😲', '😮', '😍', '🤐', '🤢', '😡', '😤', '😳', '😮', '😂', '😳', '🙀', '😭', '😮', '💥', '😠', '😁', '😂', '😳', '😍', '😲', '😲', '🤩', '😁', '👍', '😲', '😊', '😒', '😂', '😁', '😉', '😱',

# 2. Keeping emojis and emoticons then transforming them into their word form


In [694]:
emoticon_to_word = {
    ":)": "happy ",
    ":]": "happy ",
    "=)": "happy ",
    ":v": "happy ",
    ':")': "huhu ",
    ":D": "laugh ",
    ":(": "sad ",
    ":<": "sad ",
    ";)": "wink ",
    ":|": "neutral ",
    ":P": "tongue ",
    ":'(": "cry ",
    ":O": "surprise ",  
    "^^": "happy ",
    "<3": "love ",
    "-.-": "annoyance ",
    ":-)": "happy ",
    "@@": "ngạc nhiên ",
    "-_-": "bored ",
    ":>": "smirking ",
    ":3": "cuteness ",
    ":'<": "sad ",
    "^^": "happy ",
    "=]": "friendly ",
    "=.=": "disbelief ",
    ":x": "secrecy ",
    "0 . 0": "shock ",
    ":#": "quiet ",
    ">.<": "frustration ",
    ":)": "khinh bỉ ",
    "☻": "cười ",
    "☺": "cười ",
    "=)": "cười ",
    ":‑)": "cười ",
    ":)": "cười ",
    ":-]": "cười ",
    ":>": "cười ",
    "8-)": "cười ",
    "8)": "cười ",
    ":-}": "cười ",
    ":}": "cười ",
    ":o)": "cười ",
    ":c)": "cười",
    ":^)": "cười ",
    "=]": "cười ",
    "=)" : "cười ",
    ":-)": "cười ",
    ":(": "buồn ",
    ":c": "buồn ",
    "-.-": "bó tay ",
    "- _ -": "bó tay ",
    "xD": "cười ",
    "XD": "cười ",
    ":V": "cười ",
    ":3": "hm ",
    ":<": "buồn ",
    ">:(": "tức ",
    ">:[": "tức ",
    ":'(": "khóc ",
    "• 3 •": "cute ",
    "xd": "tức ",
    "uwu": ":cute ",
    "UwU": "cute ",
    "owo": "cute face ",
    "OwO": "cute face ",
    

}

In [695]:

def transform_emoticons(text):
    for emoticon, word in emoticon_to_word.items():
        text = text.replace(emoticon, word)
    return text

def transform_emojis_and_emoticons(text):
    text_with_emojis = emoji.demojize(text)
    text_with_words = transform_emoticons(text_with_emojis)
    return text_with_words

# Example usage
input_text = "I'm feeling so happy! 😃 :D ;)"
output_text = transform_emojis_and_emoticons(input_text)
print(output_text)
df['Sentence'] = df['Sentence'].apply(transform_emojis_and_emoticons)
df

I'm feeling so happy! :grinning_face_with_big_eyes: laugh  wink 


,Emotion,Sentence
0,Other,1423k cho mình xin :grinning_face_with_big_eye...
1,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2,Disgust,lo học đi . yêu đương lol gì hay lại thích học...
3,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này cười
4,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...
...,...,...
12340,Surprise,Tôi không bao giờ từng nghĩ rằng bạn sẽ đạt đư...
12341,Surprise,Tôi cảm thấy thực sự vui mừng với sự thành côn...
12342,Surprise,Tôi không thể nghĩ ra một cách nào để mô tả cả...
12343,Surprise,Tôi thật sự rất ngạc nhiên với điều đó.


# 3. Translate emoji and emoticons

In [696]:
english_to_vietnamese = {
    "happy": "vui vẻ",
    "laugh": "cười",
    "sad": "buồn",
    "wink": "nháy mắt",
    "neutral": "trung lập",
    "tongue": "trêu đàu",
    "cry": "khóc",
    "surprise": "ngạc nhiên",
    "love": "yêu",
    "annoyance": "tức giận",
    "confusion": "bất ngờ",
    "bored": "chán nản",
    "smirking": "nhếch môi",
    "cuteness": "đáng yêu",
    "friendly": "thân thiện",
    "disbelief": "hoài nghi",
    "secrecy": "bí mật",
    "shock": "shock",
    "quiet": "im lặng",
    "frustration": "thất vọng",

    
}

In [697]:
def translate_english_to_vietnamese(text):
    for english, vietnamese in english_to_vietnamese.items():
        text = text.replace(english, vietnamese)
    return text

# Example usage
input_text = "Tôi rất hạnh phúc! 😃😃😃:D ;)"
output_text = transform_emojis_and_emoticons(input_text)
print(translate_english_to_vietnamese(output_text))
df['Sentence'] = df['Sentence'].apply(translate_english_to_vietnamese)
df

Tôi rất hạnh phúc! :grinning_face_with_big_eyes::grinning_face_with_big_eyes::grinning_face_with_big_eyes:cười  nháy mắt 


,Emotion,Sentence
0,Other,1423k cho mình xin :grinning_face_with_big_eye...
1,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2,Disgust,lo học đi . yêu đương lol gì hay lại thích học...
3,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này cười
4,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...
...,...,...
12340,Surprise,Tôi không bao giờ từng nghĩ rằng bạn sẽ đạt đư...
12341,Surprise,Tôi cảm thấy thực sự vui mừng với sự thành côn...
12342,Surprise,Tôi không thể nghĩ ra một cách nào để mô tả cả...
12343,Surprise,Tôi thật sự rất ngạc nhiên với điều đó.


In [698]:
list_translate_emojis =[]
for i in individual_emojis:
    c = transform_emojis_and_emoticons(i)
    list_translate_emojis.append(c)
    print(c)

:face_with_steam_from_nose:
:pouting_face:
:face_with_symbols_on_mouth:
:kissing_face:
:kissing_face_with_closed_eyes:
:water_wave:
:disappointed_face:
:slightly_frowning_face:
:fearful_face:
:face_with_open_mouth:
:face_with_open_mouth:
:star-struck:
:airplane:
:beaming_face_with_smiling_eyes:
:smiling_face_with_heart-eyes:
buồn ollision:
:beaming_face_with_smiling_eyes:
:face_screaming_in_fear:
:astonished_face:
:smiling_face_with_heart-eyes:
:two_hearts:
:smiling_face_with_heart-eyes:
:exploding_head:
buồn urly_hair:
:face_screaming_in_fear:
:exploding_head:
:face_screaming_in_fear:
:face_with_tears_of_joy:
buồn ooked_rice:
:tired_face:
:face_with_raised_eyebrow:
:shushing_face:
:face_with_open_mouth:
:smiling_face_with_smiling_eyes:
:frowning_face:
:smiling_face_with_heart-eyes:
:star-struck:
:astonished_face:
:star-struck:
:smiling_face_with_hearts:
:astonished_face:
:partying_face:
:anxious_face_with_sweat:
:broken_heart:
:face_screaming_in_fear:
:flushed_face:
:thinking_face:
:b

In [700]:
translate_emoticons_to_Vietnamese = {
    ":pouting_face:": "tức giận ",
    ":disappointed_face:": "buồn " ,
    ":face_with_tears_of_joy:": "haha ",
    ":beaming_face_with_smiling_eyes:": "vui ",
    ":smiling_face_with_smiling_eyes:": "vui ",
    ":loudly_crying_face:": "buồn quá ",
    ":expressionless_face:": "thất vọng ",
    ":hot_face:": "mệt ",
    ":rolling_on_the_floor_laughing:": "vui ",
    ":smiling_face_with_sunglasses:": "ngầu ",
    ":face_blowing_a_kiss:": "đáng yêu ",
    ":face_with_rolling_eyes:": "kinh ",
    ":grinning_face_with_sweat:": "ngại nhỉ ",
    ":grinning_face:": "hớn hở ",
    ":smiling_face_with_heart-eyes:": "đáng yêu ",
    ":red_heart:": "yêu yêu ",
    ":slightly_frowning_face:": "buồn quá ",
    ":pensive_face:": "chán nhỉ ",
    ":weary_face:": "mệt mỏi ",
    ":face_with_raised_eyebrow:": "suy nghĩ ",
    ":grinning_squinting_face:": "haha ",
    ":sleepy_face:": "mệt ",
    ":crying_face:": "buồn ",
    "surpriseK_hand:": "tốt ",
    ":smirking_face:": "khinh bỉ ",
    ":relieved_face:":"hihi ",
    ":face_vomiting:": "kinh tởm ",
    ":smiling_face_with_horns:":"kinh tởm ",
    ":face_with_steam_from_nose:": "bực tức ",
    ":face_with_monocle:": "lo lắng ",
    ":angry_face:": "tức ",
    ":smiling_face:": "vui",
    ":kissing_face_with_closed_eyes:": "yêu ",
    ":smiling_face_with_hearts:": "vui ",
    ":face_savoring_food:": "ngon ",
    ":confused_face:": "khó chịu ",
    ":thumbs_down:": "ghét ",
    ":money-mouth_face:": " ",
    ":cherry_blossom:": " ",
    ":grimacing_face:": "khó chịu ",
    ":spider_web:": " ",
    ":dog:": " ",
    ":fearful_face:": "sợ hãi ",
    ":grinning_face_with_smiling_eyes:": "cười ",
    ":downcast_face_with_sweat:": "mệt mỏ ",
    ":star-struck:": "thích ",
    ":folded_hands:": "hy vọng ",
    ":face_with_head-bandage:": "đau mệt ",
    ":pile_of_poo:": "cứt ",
    ":sleeping_face:": "ngủ ",
    ":smiling_cat_with_heart-eyes:": "yêu ",
    ":shushing_face:": "im lặng ",
    ":tired_face:": "mệt",
    ":face_with_hand_over_mouth:": "ngại ngùng ",
    ":new_moon_face:": "lạc quan ",
    ":frowning_face:": "buồn ",
    ":thumbs_up:": "thích ",
    ":flexed_biceps:": "cố gắng ",
    ":full_moon_face:": "hihi ",
    ":sad_but_relieved_face:": "buồn ",
    ":clapping_hands:": "chúc mừng ",
    ":ghost:": "kinh tởm ",
    ":person_facepalming:": "thất vọng ",
    ":light_skin_tone:": " ",
    ":anxious_face_with_sweat:": "lo lắng ",
    ":woman_dancing:": " ",
    "happyictory_hand:": "hạnh phúc ",
    ":index_pointing_up:": "ở trên ",
    ":persevering_face:": "cố gắng ",
    ":winking_face:": "ok ",
    ":female_sign:": " ",
    ":male_sign:": " ",
    ":face_without_mouth:": "bình thường ",
    ":unamused_face:": "tức ",
    ":face_with_open_mouth:": "ngạc nhiên ",
    ":winking_face_with_tongue:": "vui ",
    ":slightly_smiling_face:": "dễ thương ",
    ":neutral_face:": "khó chịu ",
    ":person_pouting:": "khinh thường ",
    ":kissing_face:": "yêu ",
    ":face_screaming_in_fear:": "kinh hãi ",
    ":drooling_face:": "ghiền quá ",
    ":squinting_face_with_tongue:": "mỉa mai ",
    ":kiss_mark:": "yêu ",
    ":two_hearts:": "thích ",
    ":sneezing_face:": "mệt ",
    ":grinning_face_with_big_eyes:": "sướng ",
    ":upside-down_face:": "ngốc ",
    ":anguished_face:": "đau đớn ",
    ":beating_heart:": "rung động ",
    ":confounded_face:": "hoang mang ",
    ":cat_with_tears_of_joy:": "vui ",
    ":smiling_face_with_open_hands:": "hạnh phúc ",
    ":face_with_symbols_on_mouth:" : "tức giận",
    ":smiling_face_with_halo:" : "ngây thơ",
    ":exploding_head:" : "hoảng loạn",
    ":bat:": " ",
    ":raising_hands:"  : " ",
    ":astonished_face:" : "ngạc nhiên thật",
    ":flushed_face:" : "ngại  ngùng",
    ":superhero:" : " ",
    ":face_with_crossed-out_eyes:" : "cười trong lo sợ",
    ":face_with_open_mouth:" : "bất ngờ",
    ":yawning_face:" : "buồn ngủ",
    ":squinting_face_with_tongue:" : "buồn",
    ":cry:" : "khóc",
    ":frowning_face:" : "buồn quá đi",
    ":hear-no-evil_monkey:" : " ",
    ":partying_face:" : "vui vẻ",
    ":zany_face:" : "kì cục",
    ":thinking_face:": "lo lắng",
    ":sun_behind_large_cloud:": " ",
    ":smiling_face_with_smiling_eyes:": "vui quá",
    ":artist_palette:": " ",
    ":skull:": "chết",
    ":hushed_face:": "im lặng",
    ":thought_balloon:": " ",
    ":hamburger:": " ",
    ":party_popper:": " ",
    ":prohibited:": " ", 
    ":broken_heart:": " đau lòng",
    ":bug:": " ", 
    ":oncoming_fist:" : " ",
    ":speak-no-evil_monkey:" : " ",
    ":weary_cat:" : "mệt mỏi",
    ":person_shrugging:" : " ",
    ":fire:" : " ",
    ":globe_showing_Europe-Africa:" : " ",
    ":high_voltage:" : " ",
    ":heart_suit:" : " ",
    ":bomb:": " ",
    ":worried_face:": "lo lắng",
    ":water_wave:" : " ",
    ":see-no-evil_monkey:" : " ",
    ":speaking_head:" : "nhiều chuyện ",
    ":birthday_cake:" : " ",
    ":boy:" : " ",
    ":girl:" : " ",
    ":snake:" : " ",
    ":sun_with_face:" : "vui vẻ ",
    ":fork_and_knife_with_plate:" : "ăn ",
    ":face_with_tongue:" : " ",
    ":tornado:" : " ",
    ":bouquet:" : " ",
    ":lion:": " ",
    ":sparkling_heart:": "hạnh phúc",
    ":nauseated_face:" :"hạnh phúc", 
    ":television:" : " ",
    ":house:" : " ",
    ":woozy_face:" : " ",
    ":soft_ice_cream:" : " ",
    ":pizza:" : " ",
    ":owl:" : " ",
    ":mobile_phone:" : " ",
    ":milky_way:" : " ",
    ":glowing_star:" : " ",
    ":person_running:" : " ",
    ":sleeping_face:" : " mệt",
    ":nerd_face:": "học sinh giỏi", 
    ":new_moon:" : " ",
    ":skull_and_crossbones:" : " ",
    ":wrapped_gift:" : " ",
    ":face_with_thermometer:" : "nóng giận",
    ":zzz:" : " ",
    ":persevering_face:" : "kiên nhẫn",
    ":muted_speaker:" : " ",
    ":fog:" : " ",
    ":world_map:" : " ",
    ":trophy:" : " ",
    ":litter_in_bin_sign:" : " ",
    ":person_gesturing_NO:" : " ",
    ":sunrise_over_mountains:" : " ",
    ":baby:" : " ",
    ":door:" : " ",
    ":spider:" : " ",
    ":stop_sign:" : " ",
    ":revolving_hearts:" : " ",
    ":police_car_light:" : " ",
    ":microphone:" : " ",
    ":zipper-mouth_face:" : "im lặng",
    ":evergreen_tree:" : " ",
    ":derelict_house:" : " ",
    ":wilted_flower:" : " ",
    ":full_moon:" : " ",
    ":speech_balloon:" : " ",
    ":person_walking:" : " ",
    ":tomato:" : " ",
    ":dashing_away:" : " ",
    ":running_shoe:" : " ",
    ":pleading_face:" : " ",
    ":rainbow:" : " ",
    ":raised_hand:" : " ",
    ":face_with_medical_mask:" : " ",
    ":zombie:" : " ",
    ":sweat_droplets:" : " ",
    ":airplane:" : " ",
    ":anger_symbol:" : "tức giận", 
    ":dog_face:" : " ",
    ":grinning_cat_with_smiling_eyes:" : " ",
    ":brick:" : " ",
    ":dizzy:" : " ",
    ":sun_behind_small_cloud:": " ",
    ":person:" : " ",

}

In [701]:
def translate_english_to_vietnamese_emoticon(text):
    for word_form, vietnamese in translate_emoticons_to_Vietnamese.items():
        text = text.replace(word_form, vietnamese)
    return text

# Example usage

input_text = ":relieved_face::relieved_face::D Tôi rất hạnh phúc! :beaming_face_with_smiling_eyes::beaming_face_with_smiling_eyes:: :D ;)"
output_text = translate_english_to_vietnamese_emoticon(input_text)
print(translate_english_to_vietnamese(output_text))
df['Sentence'] = df['Sentence'].apply(translate_english_to_vietnamese_emoticon)
df

hihi hihi :D Tôi rất hạnh phúc! vui vui : :D ;)


,Emotion,Sentence
0,Other,1423k cho mình xin sướng bài nhạc tên là gì với ạ
1,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2,Disgust,lo học đi . yêu đương lol gì hay lại thích học...
3,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này cười
4,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...
...,...,...
12340,Surprise,Tôi không bao giờ từng nghĩ rằng bạn sẽ đạt đư...
12341,Surprise,Tôi cảm thấy thực sự vui mừng với sự thành côn...
12342,Surprise,Tôi không thể nghĩ ra một cách nào để mô tả cả...
12343,Surprise,Tôi thật sự rất ngạc nhiên với điều đó.


# 4. Thay thế những từ bị gõ sai và sai chính tả.

In [702]:
def replace_words(text):
    correct_words = {
        "quáa": "quá",
        "coá": "có",
        "ngta": "người ta",
        "nta": "người ta",
        "cf": "cà phê",
        "coffee": "cà phê",
        "cafe": "cà phê ",
        "caphe": "cà phê",
        "coffe": "cà phê",
        "hk": "không",
        "cũnh": "cũng",
        "cungc": "cũng",
        "cungz": "cũng",
        "pik": "biết",
        "pk": "biết",
        "bik": "biết",
        "bjt": "biết",
        "nge": "nghe",
        "t": "tui",
        "uh": "ừm",
        "uhm": "ừm",
        "ah": "à",
        "v": "vậy",
        "thậc": "thật",
        "thixx": "thích",
        "nhuwnh": "nhưng",
        "mợt": "mệt",
        "tnao": "thế nào",
        "thik": "thích",
        "k": "không",
        "truất": "chất",
        "fa": "Forever Alone",
        "h": "giờ",
        "chời": "trời",
        "lun": "luôn",
        "trym": "chim",
        "dume": "đụ mẹ",
        "lol": "lồn",
        "loz": "lồn",
        "mừ": "mà",
        "cmnl": "con mẹ nó luôn",
        "đm": "đụ mẹ",
        "ml": "mặt lồn",
        "đcm": "địt cụ mày",
        "zai": "trai",
        "vkl": "vãi cả lồn",
        "thoi": "thôi",
        "mọe": "mẹ",
        "âu kề": "ok",
        "gét": "ghét",
        "ah": "à",
        "vailon": "vãi lồn",
        "đựu": "đụ",
        "lìn": "lồn",
        "gê": "ghê",
        "douma": "đụ má",
        "lòn":"lồn",
        "tềnh": "tình",
        "nthe": "như thế",
        "bều": "bèo",
        "chụy": "chị",
        "mòa": "mà",
        "th": "thôi",
        "v": "vậy",
        "b": "bạn",
        "ny": "người yêu",
        "đbh": "đéo bao giờ",
        "wtf": "what the fuck",
        "rùi": "rồi",
        "gvcn": "giáo viên chủ nhiệm",
        "ròi": "rồi",
        "z": "vậy",
        "àh": "à",
        "rụg": "rụng",
        "vạy": "vậy",
        "nv": "nhân viên",
        "ròi": "rồi",
        "đóa": "đó",
        "vid": "video",
        "hk": "không",
        "clm": "cái lồn má",
        "vlon": "vãi lồn",
        "mềnh": "mình",
        "cmt": "bình luận" ,
        "hỏg": "hỏng",
        "ẻ": "ỉa",
        "dồi": "rồi",
        "dth": "dễ thương",
        "nge": "nghe",
        "ak": "á",
        "iêu": "yêu",
        "thíu": "thiếu",
        "m": "mầy",
        "cíu": "cứu",
        "n": "nó",
        "móa": "má",
        "thặc": "thật",
        "tks": "cảm ơn",
        "c": "chị",
        "mé": "má",
        "pê": "bê",
        "rứa": "thế",
        "hỉ": "hả",
        "stt": "bài đăng",
        "zời": "trời",
        "zo": "giờ",
        "lsao": "làm sao",
        "móa": "má",
        "phe": "phê",
        "r": "rồi",
        "ah": "à",
        "pv": "phục vụ",
        "dth" : "dễ thương",
        "hloz": "hãm lồn",
        "cc": "anh chị",
        "zay": "vậy",
        "hvl": "huấn luyện viên",
        "kara": "karaoke",
        "vại": "vậy",
        "kau": "tôi",
        "co": "có",
        "saolol" : "xạo lồn",
        "bn": "bạn",
        "đzai" : "đẹp trai",
        "iêu": "yêu",
        "thíu": "thiếu",
        "ghe": "ghê",
        "ngulon": "ngu lồn",
        "Kbh": "không bao giờ",
        "mềnh" : "mình",
        "đag": "đang",
        "aw" : "á",
        "cmt": "bình luận",
        "ctay": "chia tay",
        "sg": "sài gòn",
        "qtrong" : "quan trọng",
        "nghr" : "nghe",
        "đg": "đường",
        "gio": "giờ",
        "anw": "ăn",
        "thâu": "thôi",
        "chời": "trời",
        "sx": "sản xuất",
        "hơm" : "không",
        "lien quan": "liên quân",
        "bth": "bình thường",
        "đíu": "đếch",
        "hỉu": "hiểu",
        "n": "nó",
        "ml": "mặt lồn",
        "xời": "trời",
        "đhs": "đếch hiểu sao",
        "v": "vậy",
        "dt": "điện thoại",
        "cta": "chúng ta",
        "post": "đăng",
        "thui": "thôi",
        "bai": "bao",
        "fa": "cô đơn",
        "Gê": "ghê",
        "lân": "lần",
        "ô": "ông",
        "ccao": "quảng cáo",
        "ưi" : "ơi",
        "hoy": "thôi",
        "ce": "chị em",
        "oy": "ôi",
        "chj": "chị",
        "chớ": "chứ",
        "quài": "hoài",
        "pê": "bế",
        "Az": "á",
        "vay": "vậy",
        "duoc": "được",
        "cung": "cung",
        "uoc": "ước",
        "đúi": "đuối",
        "mum": "mẹ",
        "mom": "mẹ",
        "bn": "bao nhiêu",
        "cđ": "cũng được",
        "chai": "trai",
        "tủi": "tuổi",
        "êu": "yêu",
        "mạ": "mẹ",
        "Ko": "không",
        "hềy": "hề",
        "đũy" : "đĩ",
        "tt": "nhắn tin",
        "quad": "quá",
        "yêuc" : "yêu",
        "ts" : "trà sữa",
        "tks": "cảm ơn",
        "dv" : "diễn viên",
        "tý": "tí",
        "bit": "biết",
        "ss": "so sánh",
        "hoi": "hỏi",
        "maiz": "mãi",
        "zai" : "trai",
        "gdinh": "gia đình",
        "disme": "đĩ mẹ",
        "yữn": "tưởng",
        "qué": "quá",
        "z": "vậy",
        "ui": "ơi",
        "kbh": "không bao giờ",
        "ngheng": "nghen",
        "móe": "mẹ",
        "mọe": "mẹ",
        "iem": "em",
        "nờ": "à",
        "hỏg": "hỏng",
        "đk": "được không",
        "thằg": "thằng",
        "hk": "không",
        "lm": "làm",
        "bđ": "bóng đá",
        "vầy": "vậy",
        "lậy": "lạy",
        "b": "bạn",
        "dị": "vậy",
        "nv": "như vậy",
        "êi": "ơi",
        "ib": "nhắn tin",
        "moiij": "mọi",    
        }
    words = text.split()
    replaced_words = [correct_words.get(word, word) for word in words]
    text = ' '.join(replaced_words)
    return text


# input_string = " t rất cóa pik a"
# replaced_string = replace_words(input_string)
# print("Original String:", input_string)
# print("Replaced String:", replaced_string)
df['Sentence'] = df['Sentence'].apply(replace_words)


# 5. Xóa stop words (và, nếu, rất, chỉ, cho,...)

In [703]:
def remove_stopwords(sentence, stopword_file='stopword.txt'):
    # Read stopwords from the file with utf-8 encoding
    with open(stopword_file, 'r', encoding='utf-8') as file:
        stopwords = set(word.strip() for word in file)

    # Tokenize the input sentence into words
    words = sentence.split()

    # Remove the stopwords from the list of words
    filtered_words = [word for word in words if word.lower() not in stopwords]

    # Reconstruct the sentence without the stopwords
    filtered_sentence = ' '.join(filtered_words)

    return filtered_sentence

# Example usage
if __name__ == "__main__":
    input_sentence = "tôi rất cần bởi cả các"
    cleaned_sentence = remove_stopwords(input_sentence)
    print("Original Sentence:", input_sentence)
    print("Cleaned Sentence:", cleaned_sentence)
df['Sentence'] = df['Sentence'].apply(remove_stopwords)


Original Sentence: tôi rất cần bởi cả các
Cleaned Sentence: tôi


In [704]:
# import string
# def remove_punctuation(sentence):
#     # Tạo một bản sao của chuỗi không chứa các dấu câu
#     cleaned_sentence = sentence.translate(str.maketrans("", "", string.punctuation))
#     return cleaned_sentence

# # Thử nghiệm hàm
# input_sentence = "Xin chào, đây là một câu ví dụ! ??? . "
# cleaned_sentence = remove_punctuation(input_sentence)
# print(cleaned_sentence)  # Kết quả: "Xin chào đây là một câu ví dụ"
# df['Sentence'] = df['Sentence'].apply(remove_punctuation)


In [705]:
def remove_double_spaces(sentence):
    cleaned_sentence = re.sub(r'\s+', ' ', sentence)
    return cleaned_sentence.strip()
print(remove_double_spaces(" Tui rất  nhớ     anh"))
df['Sentence'] = df['Sentence'].apply(remove_double_spaces)


Tui rất nhớ anh


In [706]:
df

,Emotion,Sentence
0,Other,1423k mình xin sướng bài nhạc tên ạ
1,Disgust,đáng đời con quỷ . về nhà lôi con nhà mày đánh...
2,Disgust,lo học đi . yêu đương lồn hay thích học sinh học
3,Enjoyment,uớc về già thể cụ cười
4,Enjoyment,lần video con coi đi coi hoài . cưng con quá .
...,...,...
12340,Surprise,Tôi bao giờ nghĩ bạn đạt đó.
12341,Surprise,Tôi cảm thấy thực vui mừng thành công bạn.
12342,Surprise,Tôi thể nghĩ một cách nào mô tả cảm giác mình.
12343,Surprise,Tôi thật ngạc nhiên đó.


# remove number

In [707]:

def remove_numbers(sentence):
    cleaned_sentence = re.sub(r'\d+', '', sentence)
    return cleaned_sentence
input_sentence = "There are 3 apples and 5 195K oranges on the table."
result = remove_numbers(input_sentence)
print(result)
df['Sentence'] = df['Sentence'].apply(remove_numbers)
df.head()

There are  apples and  K oranges on the table.


,Emotion,Sentence
0,Other,k mình xin sướng bài nhạc tên ạ
1,Disgust,đáng đời con quỷ . về nhà lôi con nhà mày đánh...
2,Disgust,lo học đi . yêu đương lồn hay thích học sinh học
3,Enjoyment,uớc về già thể cụ cười
4,Enjoyment,lần video con coi đi coi hoài . cưng con quá .


# Remove particular word

In [708]:
# def remove_words_from_sentence(sentence, file_path='xoa_het_1.txt'):
#     with open(file_path, 'r', encoding='utf-8') as f:
#         words_to_remove = set(line.strip() for line in f)
    
#     words = sentence.split()
#     filtered_words = [word for word in words if word not in words_to_remove]
    
#     filtered_sentence = ' '.join(filtered_words)
#     return filtered_sentence

# # Example usage
# sentence = "This is a sample sentence with some words. 20k 20 20tr ai tui ai nghe "
# filtered_sentence = remove_words_from_sentence(sentence)
# print(filtered_sentence)
# df['Sentence'] = df['Sentence'].apply(remove_words_from_sentence)


In [709]:
def convert_to_lowercase(input_text):
    return input_text.lower()
text = "Ví Dụ Về Chữ Viết THƯỜNG"
lowercased_text = convert_to_lowercase(text)
df['Sentence'] = df['Sentence'].apply(convert_to_lowercase)

print(lowercased_text)

ví dụ về chữ viết thường


In [710]:
import re

def remove_lonely_letters(input_str):
    pattern = r'\b[a-zA-Z]\b'  # Mẫu regex tìm các chữ cái đứng một mình
    result = re.sub(pattern, '', input_str)
    return result

input_sentence = "t rất bùn lun á m"
modified_sentence = remove_lonely_letters(input_sentence)
print(modified_sentence)

 rất bùn lun á 


In [711]:
df['Sentence'] = df['Sentence'].apply(remove_lonely_letters)

In [712]:

# Function to convert string data to numbers in the "label" column
def convert_string_to_numbers(df, column_name):
    df["label"] = pd.factorize(df["Emotion"])[0]

# Call the function to convert string data to numbers in the "label" column
convert_string_to_numbers(df, "Emotion")
df.head()



,Emotion,Sentence,label
0,Other,mình xin sướng bài nhạc tên ạ,0
1,Disgust,đáng đời con quỷ . về nhà lôi con nhà mày đánh...,1
2,Disgust,lo học đi . yêu đương lồn hay thích học sinh học,1
3,Enjoyment,uớc về già thể cụ cười,2
4,Enjoyment,lần video con coi đi coi hoài . cưng con quá .,2


In [713]:
df.rename(columns={'Sentence': 'text'}, inplace=True)
df.head()

,Emotion,text,label
0,Other,mình xin sướng bài nhạc tên ạ,0
1,Disgust,đáng đời con quỷ . về nhà lôi con nhà mày đánh...,1
2,Disgust,lo học đi . yêu đương lồn hay thích học sinh học,1
3,Enjoyment,uớc về già thể cụ cười,2
4,Enjoyment,lần video con coi đi coi hoài . cưng con quá .,2


In [714]:
df.drop('Emotion', axis=1, inplace=True)
df.head()

,text,label
0,mình xin sướng bài nhạc tên ạ,0
1,đáng đời con quỷ . về nhà lôi con nhà mày đánh...,1
2,lo học đi . yêu đương lồn hay thích học sinh học,1
3,uớc về già thể cụ cười,2
4,lần video con coi đi coi hoài . cưng con quá .,2


In [715]:
df = df.dropna(subset=['text'])


In [716]:
df.to_csv("balance_data_way2.csv", index=False)


In [484]:
df['label'].value_counts()

4    1837
0    1803
3    1772
5    1763
2    1751
1    1725
6    1687
Name: label, dtype: int64

In [900]:
nan_count = df['text'].isna().sum()
nan_count


0

In [604]:
import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

# Load your data from CSV or other sources
data = pd.read_csv('imbalance_data_way2.csv')

# Extract labels from the 'label' column
y = data['label'].values
x = data['text']
y = np.array(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.15, random_state=42)

# Calculate class weights based on training labels
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Print the calculated class weights
for class_id, weight in enumerate(class_weights):
    print(f"Class {class_id}: Weight {weight:.2f}")


Class 0: Weight 0.78
Class 1: Weight 0.73
Class 2: Weight 0.51
Class 3: Weight 2.13
Class 4: Weight 3.14
Class 5: Weight 0.84
Class 6: Weight 2.46
